## Python 中的 IO 多路复用

- 在并发高的情况下，连接活跃度不高，epoll 比 select 好
- 在并发性不高，同时连接很活跃时，select 比 epoll 好

- 非阻塞 IO 实现http 请求
    - 使用 while 循环等待，并没有提高并发

In [ ]:
import socket
from urllib.parse import urlparse

def get_url(url):
    # 通过 socket 请求 html
    url = urlparse(url)
    host = url.netloc
    path = url.path
    if path == "":
        path = "/"
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 申明使用非阻塞 IO
    client.setblocking(False)
    try:
        client.connect((host, 80))
    except BlockingIOError as e:
        pass

    while True:
        try:
            client.send("GET {} HTTP/1.1\r\nHost:{}\r\nConnection:close\r\n\r\n".format(path, host).encode("utf-8"))
            break
        except OSError as e:
            pass

    data = b""
    while True:
        try:
            d = client.recv(1024)
        except BlockingIOError as e:
            continue
        if d:
            data += d
        else:
            break
    data = data.decode("utf-8")
    html_data = data.split("\r\n\r\n")[1]
    print(html_data)
    client.close()

if __name__ == "__main__":
    get_url("http://www.baidu.com")


<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta content="always" name="referrer" />
    <meta
        name="description"
        content="全球领先的中文搜索引擎、致力于让网民更便捷地获取信息，找到所求。百度超过千亿的中文网页数据库，可以瞬间找到相关的搜索结果。"
    />
    <link rel="shortcut icon" href="//www.baidu.com/favicon.ico" type="image/x-icon" />
    <link
        rel="search"
        type="application/opensearchdescription+xml"
        href="//www.baidu.com/content-search.xml"
        title="百度搜索"
    />
    <title>百度一下，你就知道</title>
    <style type="text/css">
        body {
            margin: 0;
            padding: 0;
            text-align: center;
            background: #fff;
            height: 100%;
        }

        html {
            overflow-y: auto;
            color: #000;
            overflow: -moz-scrollbars;
            height: 100%;
        }

        body, input {
            font-size

- 使用 select 完成 http 请求

In [4]:
import socket
from urllib.parse import urlparse
from selectors import DefaultSelector, EVENT_READ, EVENT_WRITE

selector = DefaultSelector()
ended = False

class Fetcher:
    def get_url(self, url):
        url = urlparse(url)
        self.host = url.netloc
        self.path = url.path
        self.data = b""
        if self.path == "":
            self.path = "/"
        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client.setblocking(False)
        
        try:
            self.client.connect((self.host, 80))
        except BlockingIOError as e:
            pass
        
        # 注册
        selector.register(self.client.fileno(), EVENT_WRITE, self.connected)
    
    def connected(self, key):
        selector.unregister(key.fd)
        self.client.send("GET {} HTTP/1.1\r\nHost:{}\r\nConnection:close\r\n\r\n".format(self.path, self.host).encode("utf-8"))
        selector.register(self.client.fileno(), EVENT_READ, self.readable)
    
    def readable(self, key):
        d = self.client.recv(1024)
        if d:
            self.data += d
        else:
            selector.unregister(key.fd)
            data = self.data.decode("utf-8")
            html_data = data.split("\r\n\r\n")[1]
            print(html_data)
            self.client.close()
            global ended
            ended = True

# 自己编写事件循环
def loop():
    while not ended:
        ready = selector.select()
        for key, mask in ready:
            callback = key.data
            callback(key)
    
    

if __name__ == "__main__":
    Fetcher().get_url("http://www.baidu.com")
    loop()


<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta content="always" name="referrer" />
    <meta
        name="description"
        content="全球领先的中文搜索引擎、致力于让网民更便捷地获取信息，找到所求。百度超过千亿的中文网页数据库，可以瞬间找到相关的搜索结果。"
    />
    <link rel="shortcut icon" href="//www.baidu.com/favicon.ico" type="image/x-icon" />
    <link
        rel="search"
        type="application/opensearchdescription+xml"
        href="//www.baidu.com/content-search.xml"
        title="百度搜索"
    />
    <title>百度一下，你就知道</title>
    <style type="text/css">
        body {
            margin: 0;
            padding: 0;
            text-align: center;
            background: #fff;
            height: 100%;
        }

        html {
            overflow-y: auto;
            color: #000;
            overflow: -moz-scrollbars;
            height: 100%;
        }

        body, input {
            font-size